In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)

In [2]:
set.seed(1)

In [3]:
# valid columns when high speed
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'RoadType', 'flag')

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
CV <- function(dfx, expr, verbose=FALSE) {        
    folds <- createFolds(dfx$flag)
    
    count <- 1
    errs <- c()
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]
        fit <- rpart(expr, data=train, method="class")
        p <- predict(fit, newdata=test)
        predictedFlags <- colnames(p)[max.col(p, ties.method = "first")]
        
        if (verbose) {            
            printf("Fold%d\n", count)
            correctVsPredict(test, predictedFlags)
            count <- count + 1
            printf("\n")
        }
        
        nerr <- sum((predictedFlags == test$flag) == FALSE)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(mean(errs))
}

In [6]:
correctVsPredict <- function(test, predictedFlags) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum((predictedRedRows$flag == 'Red') == TRUE)
    nPredictedReds <- nrow(predictedRedRows)
    printf("As for A: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum((predictedBlueRows$flag == 'Blue') == TRUE)
    nPredictedBlues <- nrow(predictedBlueRows)
    printf("As for B: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)
}

In [7]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [8]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Red

In [9]:
df3 <- read.csv("../data/middle/sp2.csv", stringsAsFactors=FALSE)

In [10]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [11]:
allFeatures <- c(colnames(df3))
expr <- createFormula(invalids, allFeatures)

In [12]:
cv <- CV(df3, expr, verbose=TRUE)
printRedRatios(df3)
printf("CV value: %f", cv)

Fold1
As for A: correct/predict = 25/40 = 0.625000
As for B: correct/predict = 16/32 = 0.500000

Fold2
As for A: correct/predict = 32/50 = 0.640000
As for B: correct/predict = 14/23 = 0.608696

Fold3
As for A: correct/predict = 26/36 = 0.722222
As for B: correct/predict = 22/37 = 0.594595

Fold4
As for A: correct/predict = 28/43 = 0.651163
As for B: correct/predict = 17/30 = 0.566667

Fold5
As for A: correct/predict = 27/39 = 0.692308
As for B: correct/predict = 20/35 = 0.571429

Fold6
As for A: correct/predict = 34/55 = 0.618182
As for B: correct/predict = 10/17 = 0.588235

Fold7
As for A: correct/predict = 27/47 = 0.574468
As for B: correct/predict = 12/27 = 0.444444

Fold8
As for A: correct/predict = 27/46 = 0.586957
As for B: correct/predict = 13/28 = 0.464286

Fold9
As for A: correct/predict = 29/46 = 0.630435
As for B: correct/predict = 14/27 = 0.518519

Fold10
As for A: correct/predict = 31/41 = 0.756098
As for B: correct/predict = 21/32 = 0.656250

Red/All = 415/731 = 0.567715


In [13]:
fit3 <- rpart(expr, data=df3, method="class")

In [14]:
summary(fit3)

Call:
rpart(formula = expr, data = df3, method = "class")
  n= 731 

          CP nsplit rel error    xerror       xstd
1 0.10443038      0 1.0000000 1.0000000 0.04238594
2 0.06487342      1 0.8955696 0.9588608 0.04214999
3 0.03164557      3 0.7658228 0.8512658 0.04126210
4 0.01898734      4 0.7341772 0.8765823 0.04150707
5 0.01582278      5 0.7151899 0.9050633 0.04175584
6 0.01265823     10 0.6297468 0.8955696 0.04167604
7 0.01054852     13 0.5917722 0.9082278 0.04178175
8 0.01000000     16 0.5601266 0.9177215 0.04185743

Variable importance
            Speed       TimeHeadway     AheadDistance        RiskFactor 
               18                16                13                11 
            Pitch             Curve AccelerationSpeed            Engine 
                7                 6                 5                 4 
       DistSignal   TimeToCollision          Curve100          Curve150 
                4                 4                 3                 3 
             